# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import pandas as pd

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
#     print(root, dirs, files)
    file_path_list = glob.glob(os.path.join(root,'*'))
    break
print(len(file_path_list))

/home/workspace
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Data Modeling for the Query 1

Given the SessionId and itemInSession, the query should return the artist, song title and song's length irrespective of userId. 

So, the primary key should be:
    
    PartitionKey: sessionId
    ClusteringKey: itemInSession

This will uniquely identify each song played in the given session and item in session by userId. 


In [16]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS session_history "
query = query + "(session_id int, item_in_session int, artist text, \
                 song_title text, song_length decimal, \
                 PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [17]:
# session.execute("DROP TABLE session_history")

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_history (session_id, item_in_session, \
                  artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0],\
                                line[9], float(line[5])))
        except Exception as e:
            print(e)

In [19]:
## Add in the SELECT statement to verify the data was entered into the table

query = "SELECT artist, song_title, song_length FROM session_history WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df_ssh = pd.DataFrame(columns=['sessionId', 'itemInSession', 'artist', 'song', 'songLength'])
i = 0

for row in rows:
#     print(row)
#     print (row.session_id, row.item_in_session, row.user_id, row.artist, row.song_title, \
#           row.song_length)
    df_ssh.loc[i] = [338, 4, row.artist, row.song_title, row.song_length]
    i+=1
df_ssh

,sessionId,itemInSession,artist,song,songLength
0,338,4,Faithless,Music Matters (Mark Knight Dub),495.3073


#### Data Modeling for Query 2

Given the sessionId and userId, the query should return the artist, song title (sorted by itemInSession) and user name (first name and last name). 

So, the primary key should be:
    
    PartitionKey: (userId, sessionId)
    ClusteringKey: itemInSession

This will uniquely identify each song played in the given session and item in session by userId. However, the clustering key will sort the songs history of user by sessionId and then by the itemInSession, which satisfies the condition of sort by itemInSession. 



In [20]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_session_history "
query = query + "(user_id int, session_id int, item_in_session int,  artist text, \
                 song_title text, first_name text, last_name text, \
                 PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e) 
                    

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session_history (user_id, session_id, item_in_session,  \
                  artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]),  line[0],\
                                line[9], line[1], line[4]))
        except Exception as e:
            print(e)

In [22]:
query = "SELECT artist, item_in_session, song_title, first_name, last_name FROM user_session_history WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df_ush = pd.DataFrame(columns=[ 'sessionId', 'user_id', 'itemInSession', 'artist',  'song', 'first Name', 'last Name'])
i = 0

for row in rows:
#     print(row)
    df_ush.loc[i] = [182, 10, row.item_in_session, row.artist, row.song_title, row.first_name, row.last_name]
    i+=1

df_ush

,sessionId,user_id,itemInSession,artist,song,first Name,last Name
0,182,10,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,182,10,1,Three Drives,Greece 2000,Sylvie,Cruz
2,182,10,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,182,10,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Data Modeling for the Query 3

Given the song_title, the query should return the user name (first name and last name). 

So, the primary key should be:
    
    PartitionKey: song_title
    ClusteringKey: user_id

This will uniquely identify each song played by user, the clustering key of userId which is unique to first name and last name.

##### Note: Even if the song is played multiple times by user, it will only update the already inserted row with the same value 

In [23]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS song_history "
query = query + "(song_title text, user_id int, first_name text, last_name text, \
                 PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e) 
                    

In [24]:
# session.execute("DROP TABLE song_history")

In [25]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_history (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        try:
            session.execute(query, ( line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

In [26]:
query = "SELECT first_name, last_name FROM song_history WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df_sh = pd.DataFrame(columns=['song', 'first Name', 'last Name'])
i = 0

for row in rows:
#     print(row)
    df_sh.loc[i] =  ['All Hands Against His Own', row.first_name, row.last_name]
    i+=1

df_sh

,song,first Name,last Name
0,All Hands Against His Own,Jacqueline,Lynch
1,All Hands Against His Own,Tegan,Levine
2,All Hands Against His Own,Sara,Johnson


### Drop the tables before closing out the sessions

In [27]:
## Drop the table before closing out the sessions
query = "DROP TABLE session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE user_session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()